## GRN using scenic+
We employed the Scenic+ pipeline for GRN inference using integrated scRNA-seq and scATAC-seq data. The Scenic+ pipeline includes these steps:

1) Topic Modeling: Calculation of topics, which are coaccessibility regions, resulting in region-topic association scores.

2) Accessibility Imputation: Binarization of region-topic scores, grouping regions for each topic. This includes running differentially accessible regions across cell types and imputing access data. Notably, the average missingness in single cells drops from ~96% to ~40% after imputation, highlighting a significant source of uncertainty.

3) CisTarget: Determination of enriched motifs for each topic and differentially enriched motifs for each cell type, yielding enriched motifs and their associated genes.

4) Run Scenic+: Calculation of region-to-gene and TF to gene relationships using regression models. This step assesses the importance of regions and TFs for target genes, using imputed region counts, which could introduce errors. After calculating importance scores, only regions enriched for TF motifs are included. The pipeline then creates TF-regions-genes triples named eRegulons, resulting in a curated list of TFs, regions, and target genes.

The Scenic+ pipeline followed is based on the standard protocol as outlined in https://scenicplus.readthedocs.io/en/latest/tutorials.html.

## Imports

In [2]:
import anndata as ad 
# import dill, os
import numpy as np
import pandas as pd
work_dir = '../../output'
path_to_blacklist= '../../input/scenicplus/hg38-blacklist.v2.bed'


In [ ]:
adata_atac = anndata.read_h5ad(f'{work_dir}/scATAC/adata_atac.h5ad')

In [ ]:
adata_atac

## <a id='toc5_1_'></a>[Topic modeling](#toc0_)

### <a id='toc5_1_1_'></a>[Create cistopic object](#toc0_)

In [ ]:
# format adata to have location*obs
adata_t = adata_atac.T
adata_t.X = adata_t.X.astype(int)

In [ ]:
from pycisTopic.cistopic_class import create_cistopic_object
cistopic_obj = create_cistopic_object(
                            fragment_matrix=adata_t.X,
                            cell_names = adata_t.var.index,
                            region_names = adata_t.obs.index,
                            path_to_blacklist=path_to_blacklist,
                            project='all_donors',
                            min_frag = 10,
                            min_cell = 1,
                            split_pattern='___'
                            )
cistopic_obj.add_cell_data(adata_t.var)

In [ ]:
import pickle, os
pickle.dump(cistopic_obj,  open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'wb'))

### <a id='toc5_1_2_'></a>[Determine number of topics (~6h using 10 cpu)](#toc0_)

In [ ]:
if True: # run locally 
    cistopic_obj = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'rb'))
    from pycisTopic.cistopic_class import run_cgs_models
    models = run_cgs_models(cistopic_obj,
                        # n_topics=[2,4,10,16,32,48],
                        n_topics=[10,16,32,48],
                        n_cpu=12,
                        n_iter=500,
                        random_state=555,
                        alpha=50,
                        alpha_by_topic=True,
                        eta=0.1,
                        eta_by_topic=False,
                        save_path=None,
                        _temp_dir = None)
    if not os.path.exists(os.path.join(work_dir, 'scenicplus/scATAC/models')):
        os.makedirs(os.path.join(work_dir, 'scenicplus/scATAC/models'))

    pickle.dump(models,
                open(os.path.join(work_dir, 'scenicplus/scATAC/models/10x_pbmc_models_500_iter_LDA.pkl'), 'wb'))
else:
    !python ../scripts/scenicplus/compute_topic_number.py

In [ ]:
import pickle, os
models = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/models/10x_pbmc_models_500_iter_LDA.pkl'), 'rb'))
cistopic_obj = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'rb'))
# cistopic_obj.add_cell_data(adata_t.var) # to fix the nan issue of cell type
from pycisTopic.lda_models import *
model = evaluate_models(models,
                       select_model=48, 
                       return_model=True, 
                       metrics=['Arun_2010','Cao_Juan_2009', 'Minmo_2011', 'loglikelihood'],
                       plot_metrics=False)

In [ ]:
cistopic_obj.add_LDA_model(model)
pickle.dump(cistopic_obj,
            open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'wb'))

In [ ]:
from pycisTopic.clust_vis import *
run_umap(cistopic_obj, target='cell', scale=True)
plot_metadata(cistopic_obj, reduction_name = 'UMAP', variables = ['cell_type'])

In [ ]:
# plot_topic(cistopic_obj, reduction_name = 'UMAP', num_columns = 4)

## <a id='toc5_2_'></a>[Accessibility imputation](#toc0_)

In [ ]:
from pycisTopic.topic_binarization import *
region_bin_topics_otsu = binarize_topics(cistopic_obj, method='otsu') #puts regions into topics
region_bin_topics_top3k = binarize_topics(cistopic_obj, method='ntop', ntop = 3000)

In [ ]:
from pycisTopic.diff_features import impute_accessibility, normalize_scores, find_highly_variable_features, find_diff_features
imputed_acc_obj = impute_accessibility(cistopic_obj, selected_cells=None, selected_regions=None, scale_factor=10**6)
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)
variable_regions = find_highly_variable_features(normalized_imputed_acc_obj, plot = False)
markers_dict = find_diff_features(cistopic_obj, imputed_acc_obj, variable='cell_type', var_features=variable_regions)

In [ ]:
os.makedirs(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers'))
pickle.dump(region_bin_topics_otsu, open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/region_bin_topics_otsu.pkl'), 'wb'))
pickle.dump(region_bin_topics_top3k, open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/region_bin_topics_top3k.pkl'), 'wb'))
pickle.dump(markers_dict, open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/markers_dict.pkl'), 'wb'))

## <a id='toc5_3_'></a>[Cistarget](#toc0_)

In [ ]:
if True: # run locally
    import pyranges as pr
    from pycistarget.utils import region_names_to_coordinates
    import pickle
    import os
    region_bin_topics_otsu = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/region_bin_topics_otsu.pkl'), 'rb'))
    region_bin_topics_top3k = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/region_bin_topics_top3k.pkl'), 'rb'))
    markers_dict = pickle.load(open(os.path.join(work_dir, 'scenicplus/scATAC/candidate_enhancers/markers_dict.pkl'), 'rb'))

    region_sets = {}
    region_sets['topics_otsu'] = {}
    region_sets['topics_top_3'] = {}
    region_sets['DARs'] = {}
    for topic in region_bin_topics_otsu.keys():
        regions = region_bin_topics_otsu[topic].index[region_bin_topics_otsu[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
        region_sets['topics_otsu'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
    for topic in region_bin_topics_top3k.keys():
        regions = region_bin_topics_top3k[topic].index[region_bin_topics_top3k[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
        region_sets['topics_top_3'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
    for DAR in markers_dict.keys():
        regions = markers_dict[DAR].index[markers_dict[DAR].index.str.startswith('chr')] #only keep regions on known chromosomes
        region_sets['DARs'][DAR] = pr.PyRanges(region_names_to_coordinates(regions))

    # these datasets are publicly available
    rankings_db = os.path.join('../../scenicplus/data/', 'hg38_screen_v10_clust.regions_vs_motifs.rankings.feather')
    scores_db =  os.path.join('../../scenicplus/data/', 'hg38_screen_v10_clust.regions_vs_motifs.scores.feather')
    motif_annotation = os.path.join('../../scenicplus/data/', 'motifs-v10-nr.hgnc-m0.00001-o0.0.tbl')
    from scenicplus.wrappers.run_pycistarget import run_pycistarget
    run_pycistarget(
        region_sets = region_sets,
        species = 'homo_sapiens',
        save_path = os.path.join(work_dir, 'scenicplus/scATAC/motifs'),
        ctx_db_path = rankings_db,
        dem_db_path = scores_db,
        path_to_motif_annotations = motif_annotation,
        run_without_promoters = True,
        n_cpu = 40,
        _temp_dir = None,
        annotation_version = 'v10nr_clust',
        )
else: 
    !python scripts/run_pycistarget.py

In [ ]:
import dill
import scanpy as sc
import os
from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
from scenicplus.cistromes import merge_cistromes
from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships
from scenicplus.TF_to_gene import calculate_TFs_to_genes_relationships

_temp_dir = '/beegfs/desy/user/nourisaj/op_multiomics_grn/output/scenicplus'
n_cpu = 40


scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
# cistrome 
merge_cistromes(scplus_obj)
with open(os.path.join(work_dir, 'scenicplus/scenicplus/scplus_obj.pkl'), 'wb') as f:
            dill.dump(scplus_obj, f, protocol = -1)
print('cistrome merged')


## <a id='toc5_4_'></a>[Scenic+ main run](#toc0_)

In [ ]:
# do this before next run 
import pickle
cistopic_obj.cell_names = cistopic_obj.cell_data.obs_id.values
cistopic_obj.selected_model.cell_topic.columns = cistopic_obj.cell_names
cistopic_obj.cell_data.index = cistopic_obj.cell_data.obs_id

pickle.dump(cistopic_obj,
            open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'wb'))

In [ ]:
if True: # run locally
    import dill
    import scanpy as sc
    import os
    from scenicplus.scenicplus_class import create_SCENICPLUS_object
    import numpy as np
    from scenicplus.cistromes import merge_cistromes
    from scenicplus.enhancer_to_gene import get_search_space, calculate_regions_to_genes_relationships
    from scenicplus.TF_to_gene import calculate_TFs_to_genes_relationships

    _temp_dir = '/beegfs/desy/user/nourisaj/op_multiomics_grn/output/scenicplus'
    n_cpu = 40


    adata = sc.read_h5ad(f'{work_dir}/scenicplus/scRNA/adata_rna.h5ad')
    cistopic_obj = dill.load(open(os.path.join(work_dir, 'scenicplus/scATAC/cistopic_obj.pkl'), 'rb'))
    menr = dill.load(open(os.path.join(work_dir, 'scenicplus/scATAC/motifs/menr.pkl'), 'rb'))
    scplus_obj = create_SCENICPLUS_object(
        GEX_anndata = adata,
        cisTopic_obj = cistopic_obj,
        menr = menr
    )
    scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
    # cistrome 
    merge_cistromes(scplus_obj)
    with open(os.path.join(work_dir, 'scenicplus/scenicplus/scplus_obj.pkl'), 'wb') as f:
                dill.dump(scplus_obj, f, protocol = -1)
    print('cistrome merged')
    # regression: region to gene  
    biomart_host = "http://sep2019.archive.ensembl.org/" 
    species='hsapiens'
    assembly='hg38'
    upstream = [1000, 150000]
    downstream = [1000, 150000]
    get_search_space(scplus_obj,
                        biomart_host = biomart_host,
                        species = species,
                        assembly = assembly, 
                        upstream = upstream,
                        downstream = downstream) 
    calculate_regions_to_genes_relationships(scplus_obj, 
                            ray_n_cpu = n_cpu, 
                            _temp_dir = _temp_dir,
                            importance_scoring_method = 'GBM')
    print('number of genes in gene space: ',scplus_obj.uns['search_space'].Gene.unique().shape)
    print('number of genes in region-to-gene: ', scplus_obj.uns['region_to_gene'].target.unique().shape)
    with open(os.path.join(work_dir, 'scenicplus/scenicplus/scplus_obj.pkl'), 'wb') as f:
                dill.dump(scplus_obj, f, protocol = -1)
    print('region to gene calculated')
    # regression: tf to gene
    calculate_TFs_to_genes_relationships(scplus_obj, 
                        tf_file = './output/utoronto_human_tfs_v_1.01.txt',
                        ray_n_cpu = n_cpu, 
                        method = 'GBM',
                        _temp_dir = _temp_dir,
                        key= 'TF2G_adj')
    print('number of TFs in TF2G_adj: ', scplus_obj.uns['TF2G_adj'].TF.unique().shape)
    print('number of genes in TF2G_adj: ', scplus_obj.uns['TF2G_adj'].target.unique().shape)
    with open(os.path.join(work_dir, 'scenicplus/scplus_obj.pkl'), 'wb') as f:
                dill.dump(scplus_obj, f, protocol = -1)
    print('tf gene calculated')
    # eRegulon formation 
    from scenicplus.grn_builder.gsea_approach import *
    from scenicplus.utils import *
    from scenicplus.eregulon_enrichment import *

    default_params = dict(min_target_genes = 10,
            adj_pval_thr = 1,
            min_regions_per_gene = 0,
            quantiles = (0.85, 0.90, 0.95),
            top_n_regionTogenes_per_gene = (5, 10, 15),
            top_n_regionTogenes_per_region = (),
            binarize_using_basc = True,
            rho_dichotomize_tf2g = True,
            rho_dichotomize_r2g = True,
            rho_dichotomize_eregulon = True,
            rho_threshold = 0.05,
            keep_extended_motif_annot = True,
            merge_eRegulons = True, 
            order_regions_to_genes_by = 'importance',
            order_TFs_to_genes_by = 'importance',
            key_added = 'eRegulons',
            cistromes_key = 'Unfiltered',
            disable_tqdm = False,
            ray_n_cpu = n_cpu,
            _temp_dir = _temp_dir)
    alt_params = dict(min_target_genes = 0,
            adj_pval_thr = 1,
            min_regions_per_gene = 0,
            quantiles = (0.7, 0.80, 0.90),
            top_n_regionTogenes_per_gene = (10, 15, 25),
            top_n_regionTogenes_per_region = (),
            binarize_using_basc = True,
            rho_dichotomize_tf2g = True,
            rho_dichotomize_r2g = True,
            rho_dichotomize_eregulon = True,
            rho_threshold = 0,
            keep_extended_motif_annot = True,
            merge_eRegulons = True, 
            order_regions_to_genes_by = 'importance',
            order_TFs_to_genes_by = 'importance',
            key_added = 'eRegulons',
            cistromes_key = 'Unfiltered',
            disable_tqdm = False, 
            ray_n_cpu = n_cpu,
            _temp_dir = _temp_dir)
    
    build_grn(scplus_obj, **alt_params)
    format_egrns(scplus_obj,
                eregulons_key = 'eRegulons',
                TF2G_key = 'TF2G_adj',
                key_added = 'eRegulon_metadata')
    with open('scplus_obj_1.pkl', 'wb') as f:
        dill.dump(scplus_obj, f, protocol = -1)
    print('e regulon finished')

else: # on server
    !python scenicplus/run_grn.py
        

## Formatize the inferred GRN

In [ ]:
import pickle
with open(f'{work_dir}/infer/scenicplus/grn/scplus_obj.pkl', 'rb') as scplus_obj:
    data = pickle.load(scplus_obj)

In [3]:
import dill
work_dir = '../../output'
scplus_obj = dill.load(open(f'{work_dir}/infer/scenicplus/grn/scplus_obj.pkl', 'rb'))

: 

In [4]:
scplus_obj

SCENIC+ object with n_cells x n_genes = 25551 x 22787 and n_cells x n_regions = 25551 x 134379
	metadata_regions:'Chromosome', 'Start', 'End', 'Width', 'cisTopic_nr_frag', 'cisTopic_log_nr_frag', 'cisTopic_nr_acc', 'cisTopic_log_nr_acc'
	metadata_genes:'n_cells'
	metadata_cell:'GEX_cell_type', 'GEX_donor_id', 'GEX_n_genes', 'GEX_louvain', 'ACC_cisTopic_nr_frag', 'ACC_cisTopic_log_nr_frag', 'ACC_cisTopic_nr_acc', 'ACC_cisTopic_log_nr_acc', 'ACC_sample_id', 'ACC_obs_id', 'ACC_cell_type', 'ACC_donor_id'
	menr:'CTX_topics_otsu_All', 'CTX_topics_otsu_No_promoters', 'DEM_topics_otsu_All', 'DEM_topics_otsu_No_promoters', 'CTX_topics_top_3_All', 'CTX_topics_top_3_No_promoters', 'DEM_topics_top_3_All', 'DEM_topics_top_3_No_promoters', 'CTX_DARs_All', 'CTX_DARs_No_promoters', 'DEM_DARs_All', 'DEM_DARs_No_promoters'
	dr_cell:'GEX_X_diffmap', 'GEX_X_draw_graph_fr', 'GEX_X_pca', 'GEX_X_umap'

In [3]:
print('eRegulon: ', len(scplus_obj.uns['eRegulons']), 
      ', TFs: ', scplus_obj.uns['eRegulon_metadata'].TF.unique().size,
      ', target genes: ', scplus_obj.uns['eRegulon_metadata'].Gene.unique().size,
      ', target region: ', scplus_obj.uns['eRegulon_metadata'].Region.unique().size) 

KeyError: 'eRegulons'

In [6]:
# save the net
scenicplus_grn = scplus_obj.uns['eRegulon_metadata'][['TF', 'Gene', 'TF2G_rho', 'Region']].drop_duplicates().reset_index(drop=True)
scenicplus_grn = scenicplus_grn[scenicplus_grn.TF2G_rho!=0]
scenicplus_grn.columns = ['source', 'target', 'weight', 'region']
scenicplus_grn.to_csv(f'{work_dir}/scenicplus/grn/grn_extended.csv')

KeyError: 'eRegulon_metadata'

: 

In [ ]:
# save TFs enriched in cistarget and remaining CIS in cistrome
cistromes = np.asarray(list(scplus_obj.uns['Cistromes']['Unfiltered'].keys()))
tfs_cistromes = np.unique(([cistrome.split('_')[0] for cistrome in cistromes]))
np.savetxt(ff'{work_dir}/scenicplus/grn/tfs_cistrome.txt',tfs_cistromes,  fmt='%s')

In [ ]:
# Number of TFs removed during motif enrichment analysis
all_tfs = np.loadtxt(f'{work_dir}/scenicplus/utoronto_human_tfs_v_1.01.txt', dtype=str)
tfs_cistromes = np.loadtxt(ff'{work_dir}/scenicplus/grn/tfs_cistrome.txt', dtype=str)
print(f'from an initial {len(all_tfs)} TFs, {len(np.intersect1d(all_tfs, tfs_cistromes))} are in cistromes')

## Some more analysis


In [21]:
tag = '_short'

In [22]:
scenicplus_grn =  pd.read_csv(f'{work_dir}/scenicplus/grn/grn_extended{tag}.csv', index_col=0)
peak_gene_scenicplus_n = scenicplus_grn.groupby('target').apply(lambda df:df['region'].shape[0])
print('max regions per gene', np.max(peak_gene_scenicplus_n.values))
print('median region per gene', np.median(peak_gene_scenicplus_n.values))

max regions per gene 96
median region per gene 7.0


In [23]:
print('number of TFs ', scenicplus_grn.source.unique().shape[0], ' CIS ', scenicplus_grn.region.unique().shape[0], ' gene ', scenicplus_grn.target.unique().shape[0])
print('number of DORC genes with t of 10 ', (peak_gene_scenicplus_n.values > 10).sum())
print('number of DORC genes with t of 5 ', (peak_gene_scenicplus_n.values > 5).sum())

number of TFs  33  CIS  25082  gene  7461
number of DORC genes with t of 10  2949
number of DORC genes with t of 5  4288


In [24]:
aa = plt.hist(peak_gene_scenicplus_n.values, bins=100)


NameError: name 'plt' is not defined

In [25]:
# direct TF-gene links
sp_grn = scenicplus_grn[['source','target','weight']].drop_duplicates(ignore_index=True)


In [26]:
sp_grn.source.unique().shape

(33,)

In [27]:
sp_grn.to_csv(f'{work_dir}/scenicplus/grn/scenicplus_grn{tag}.csv')

In [ ]:
# cp 
sp_ct = pd.read_csv(f'{work_dir}/infer/scenicplus/grn/grn_extended.csv', index_col=0)[['region', 'target']].drop_duplicates()
sp_ct.columns = ['CRE','target']